In [1]:
import spacy, string, pandas
from spacy.vocab import Vocab
nlp = spacy.load('de_core_news_sm')
from time import time

In [2]:
# LOADS DATA
paths = ['_in_v2', '_1norm', '_1Snorm', '_2norm', '_2Snorm']
path = './IO_YO/all' + paths[4]
input_list = open(path + '.csv', encoding = 'utf-8').read().replace('\"', '').replace('\ufeff', '').split('\n')
input_table = [row.split(';') for row in input_list]

# CREATE AGGREGATION LEVELS
ids = []
ages = []
for i in range(0, len(input_table)-1):
    if input_table[i][1] not in ids:
        ids.append(input_table[i][1])
    if input_table[i][2] not in ages:
        ages.append(input_table[i][2])
        
print('input table for ' + path + ': done')

input table for ./IO_YO/all_2Snorm: done


In [3]:
# EXTRACT AND TRANSFORM TRANSCRIPTIONS
t = time()
column1 = [row[0] for row in input_table]
column1_tokenlist = [nlp(row) for row in column1]
print('1st column tokenization: done in %s s' %round((time() - t), 5))

column1_stringlist = []
for row in column1_tokenlist:
    row_stringlist = [token.text for token in row]
    column1_stringlist.append(row_stringlist)

for i in range (0, len(input_table)-1):
    input_table[i][0] = column1_stringlist[i]
print('1st column string conversion: done')

1st column tokenization: done in 69.03111 s
1st column string conversion: done


In [4]:
# INIT VOCABULARY
vocabulary = []
for a in range (0, len(ages)):
    vocabulary.append([ages[a], 0, set()]) 
for i in range (0, len(ids)):
    vocabulary.append([ids[i], 0, set()])
vocabulary.append(['all', 0, set()])

# FILL IN VOCABULARY
for i in range(0, len(input_table)-1):
    
    # age-level calculation
    for j in range (0, len(ages)):
        if input_table[i][2] == ages[j]:
            for word in input_table[i][0]:
                if word.strip().lower() not in vocabulary[j][2]:
                    vocabulary[j][2].add(word.strip().lower())
                vocabulary[j][0] = ages[j]
                vocabulary[j][1] = len(vocabulary[j][2])
                
    # person-level calculation             
    for k in range (0, len(ids)):                
        if input_table[i][1] == ids[k]:
            for word in input_table[i][0]:
                if word.strip().lower() not in vocabulary[len(ages) + k][2]:
                    vocabulary[len(ages) + k][2].add(word.strip().lower())  
                vocabulary[len(ages) + k][0] = ids[k]
                vocabulary[len(ages) + k][1] = len(vocabulary[len(ages) + k][2])
                
    # all-level calculation  
    for word in input_table[i][0]:
        if word.strip().lower() not in vocabulary[len(vocabulary)-1][2]:
            vocabulary[len(vocabulary)-1][2].add(word.strip().lower())
        vocabulary[len(vocabulary)-1][0] = 'all'
        vocabulary[len(vocabulary)-1][1] = len(vocabulary[len(vocabulary)-1][2])

#print vocabulary
#for row in vocabulary:
#    print(row[0] + ' size: ' + str(row[1]) + ' len: ' + str(len(row[2])))  

In [5]:
# INIT WORD-FREQUENCY DICTIONARY
word_freq = []
for v in range(0, len(vocabulary)):
    tmp = []
    tmp.append(vocabulary[v][0])
    tmp.append(dict.fromkeys(vocabulary[v][2], 0))
    word_freq.append(tmp)

# FILL IN WORD-FREQUENCY DICTIONARY
for i in range(0, len(input_table)-1):
   
    # age-level calculation
    for j in range (0, len(ages)):
        if input_table[i][2] == ages[j]:
            for word in input_table[i][0]:
                word_freq[j][1][word.strip().lower()] += 1  

    # all-level calculation  
    for word in input_table[i][0]:
        word_freq[len(word_freq)-1][1][word.strip().lower()] += 1      
        

In [6]:
#TRANSFORM DICTIONARY FOR PRINTING - HELPER METHOD
for w in range (0, len(word_freq)):
    dict_to_list = ''
    for word, freq in word_freq[w][1].items():
        dict_to_list = dict_to_list + word + ': ' + str(freq) + '\n'
    word_freq[w][1] = dict_to_list


In [7]:
print(word_freq[len(word_freq)-1][1])
#df = pandas.DataFrame(word_freq).T
#print(df)

: 592
billig: 4
schieben: 6
wasservogel: 1
aufgeben: 4
charakter: 2
schwiegersohn: 1
laute: 1
herzzeitvolumen: 2
horror: 1
suppe: 8
james: 1
s5: 1
toscana: 1
invasiv: 1
vulkan: 1
herübertun: 1
rotblond: 1
kurz: 53
retten: 2
hühnerwohnung: 1
stürzen: 1
abbilden: 1
langfristig: 1
ausfahrt: 1
costa: 1
teeren: 1
überführen: 1
merkmal: 1
verschmelzen: 1
europäisch: 1
trippeln: 1
aare: 2
umfasst: 1
zusammenbasteln: 1
master: 1
ausziehen: 3
black: 3
helveter: 3
fränkchen: 1
updaten: 2
tattoo: 1
gloria: 1
jahr: 102
sektor: 5
kleidchen: 1
agressiv: 2
kreuzen: 2
freund: 15
steinwüste: 1
sabine: 1
kennenlernen: 2
abdruck: 1
verhaltensweise: 1
metzger: 2
wasser: 45
absichtlich: 2
karotte: 3
rest: 6
erfüllen: 2
mama: 12
schwul: 2
anstecken: 1
bögeln: 1
verpassen: 2
mündlich: 2
benutzen: 3
user: 1
pickieren: 1
dresden: 1
spiegelverkehrt: 1
koscher: 1
zange: 1
reden: 54
gefühl: 58
positiv: 5
spielgeschwindigkeit: 1
monatsputz: 1
prinzip: 3
intellekt: 1
wanken: 1
abwasser: 4
gl: 1
kopfhörer: 2
jane: 1

In [8]:
# save data to file
table = df
with open(path + '_stat.csv', 'w', encoding = 'utf-8') as doc_out:
    for row in table:
        for cell in row:
            doc_out.write(cell + ';')
        doc_out.write('\n')
print('save data: done')

NameError: name 'df' is not defined